<a href="https://colab.research.google.com/github/AlejandroAhogado/Proyecto1BI/blob/main/Proyecto_1_Grupo_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Importacion/Instalacion de librerias

In [ ]:
#Manejo de datos
import pandas as pd
import numpy as np
from tqdm import tqdm
import re, string, unicodedata

#Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Librerías para manejo de Lenguaje Natural
!pip install contractions --quiet
import contractions
from contractions import contractions_dict
import inflect
from wordcloud import WordCloud, STOPWORDS

# NLTK
import nltk
nltk.download('all')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

#Entrenamiento del modelo
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC

from tqdm import tqdm
import tensorflow as tf
import torch

import csv

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [ ]:
data = pd.read_csv('SuicidiosProyecto.csv',sep=',', encoding='utf-8', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


# II. Exploramiento de datos

Se hace una exploracion inicial de los datos para ver en que formato estan presentados y que tipo de pre-proesamiento hay que hacer

In [ ]:
data.head(10)

,Unnamed: 0,text,class
0,173271,i want to destroy myselffor once everything wa...,suicide
1,336321,I kinda got behind schedule with learning for ...,non-suicide
2,256637,I'm just not sure anymoreFirst and foremost: I...,suicide
3,303772,please give me a reason to liveThats too much ...,suicide
4,293747,27f struggling to find meaning moving forwardI...,suicide
5,205651,Let’s get this bread 😎 Anyone know any good ba...,non-suicide
6,97174,"Day 126 of posting random ""fun"" facts everyday...",non-suicide
7,195945,Little brother is self mutilating. Please help...,suicide
8,305273,Why do women always go in groups to their wash...,non-suicide
9,69929,Did you guys know that there's no school for g...,non-suicide


# III. Preprocesamiento de Datos

## 3.1 Transformacion columna objetivo

Inicialmente se cambia la columna objetivo a un valor binario 0/1 que corresponde a las categorias de non-suicide/suicide respectivamente. Tambien se elimina la columna "unnamed:0" pues esta corresponde a un identificador unico que no es relevante para la tarea propuesta

In [ ]:
df=data.copy()
df=df.drop(df.columns[0], axis=1)
df['class'].replace(('suicide'), ('1'), inplace=True)
df['class'].replace(('non-suicide'), ('0'), inplace=True)
df['class'] = pd.to_numeric(df['class'])
df

,text,class
0,i want to destroy myselffor once everything wa...,1
1,I kinda got behind schedule with learning for ...,0
2,I'm just not sure anymoreFirst and foremost: I...,1
3,please give me a reason to liveThats too much ...,1
4,27f struggling to find meaning moving forwardI...,1
...,...,...
195695,Drop some cool new cereal ideas Like what woul...,0
195696,Unpopular opinion but cats deserve love and re...,0
195697,Hey guys :) How yall doin?,0
195698,uhm I covered my dog in a blanket because the ...,0


Ademas se valida el contenido de la columna 'class' para verificar que no haya ninguna inconsistencia con los labels

In [ ]:
df['class'].value_counts()

0    110165
1     85535
Name: class, dtype: int64

Tambien se valida la validez y completitud de los datos verificando que no hallan nulos o duplicados y eliminando las filas correspondientes en caso que se encuentren

In [ ]:
df.duplicated().value_counts()

False    195700
dtype: int64

In [ ]:
data.isna().sum()

Unnamed: 0    0
text          0
class         0
dtype: int64

Por ultimo se verifica que las columnas tengan el tipo de dato apropiado para su procesamiento

In [ ]:
df.dtypes

text     object
class     int64
dtype: object

## 3.2 Preparacion de datos
Todos los pasos contenidos en esta seccion corresponden a la limpieza y pre proceamiento del texto en la columna 'text' para poder dejarlo en un formato apto para su posterior evaluacion con modelos de PLN. Inicialmente se separan los datos en texto y labels para poder realizar ls tareas de limpieza solo sobre los datos de texto que lo requieren



In [ ]:
_, data_small = train_test_split(df, test_size=0.20, random_state=42)

In [ ]:
train, test = train_test_split(data_small, test_size=0.2, random_state=42)
X_train, y_train = train['text'], train['class']
X_test, y_test = test['text'], test['class']
print("Tamaño del conjunto de entrenamiento: "+str(X_train.size))
print("Tamaño del conjunto de evaluación: "+str(X_test.size))

Tamaño del conjunto de entrenamiento: 31312
Tamaño del conjunto de evaluación: 7828


### 3.2.1 Eliminacion contracciones

Eliminar contracciones permite reducir el tamaño del diccionario y evita que se tomen como palabras distintas casos como, por ejemplo, won't = will not. I'm = I am

In [ ]:
X_clean= X_train.apply(contractions.fix) #Aplica la corrección de las contracciones
X_clean_test= X_test.apply(contractions.fix) #Aplica la corrección de las contracciones
X_clean

170783    I really hate my life.I am 13 years old and I ...
60450     I Am ending it tonightim leaving so many frien...
159181    Yo anyone down to chat? I am 14M, very emo, an...
63934     Someone knocked Is not it fun that one of my p...
90090     I do not know what to do and I am scared that ...
                                ...                        
161943                   i am uselesshaving a panic attack.
188694    I feel like ending it would feel better than s...
20015     Sometimes I just feel like I cannot do this an...
141904    i have lost the will to live anymore,i am goin...
127158    The release of the new Netflix film is the dra...
Name: text, Length: 31312, dtype: object

### 3.2.2 Minusculas

In [ ]:
def to_lowercase(phrase):
    new_phrase = []
    for word in phrase:
        new_word = word.lower()
        new_phrase.append(new_word)
    return "".join(new_phrase)

In [ ]:
X_clean=X_clean.apply(to_lowercase)
X_clean_test=X_clean_test.apply(to_lowercase)
X_clean

170783    i really hate my life.i am 13 years old and i ...
60450     i am ending it tonightim leaving so many frien...
159181    yo anyone down to chat? i am 14m, very emo, an...
63934     someone knocked is not it fun that one of my p...
90090     i do not know what to do and i am scared that ...
                                ...                        
161943                   i am uselesshaving a panic attack.
188694    i feel like ending it would feel better than s...
20015     sometimes i just feel like i cannot do this an...
141904    i have lost the will to live anymore,i am goin...
127158    the release of the new netflix film is the dra...
Name: text, Length: 31312, dtype: object

### 3.2.3 puntuacion

In [ ]:
def remove_punctuation(phrase):
    return phrase.translate(str.maketrans('','',string.punctuation))

In [ ]:
X_clean=X_clean.apply(remove_punctuation)
X_clean_test=X_clean_test.apply(remove_punctuation)
X_clean

170783    i really hate my lifei am 13 years old and i r...
60450     i am ending it tonightim leaving so many frien...
159181    yo anyone down to chat i am 14m very emo and i...
63934     someone knocked is not it fun that one of my p...
90090     i do not know what to do and i am scared that ...
                                ...                        
161943                    i am uselesshaving a panic attack
188694    i feel like ending it would feel better than s...
20015     sometimes i just feel like i cannot do this an...
141904    i have lost the will to live anymorei am going...
127158    the release of the new netflix film is the dra...
Name: text, Length: 31312, dtype: object

### 3.2.4 Remover stopwords

In [ ]:
def remove_stopwords(phrase, stopwords=stopwords.words('english')):
    words=phrase.split(" ")
    new_phrase = []
    for word in words:
        if word not in stopwords:
            new_phrase.append(word)
            new_phrase.append(" ")
    return "".join(new_phrase)[:-1]

In [ ]:
X_clean=X_clean.apply(remove_stopwords)
X_clean_test=X_clean_test.apply(remove_stopwords)
X_clean

170783    really hate lifei 13 years old really hate lif...
60450     ending tonightim leaving many friends pets gir...
159181    yo anyone chat 14m emo spotify addict longest ...
63934     someone knocked fun one parents knocked door e...
90090     know scared screw something bestie f really cl...
                                ...                        
161943                           uselesshaving panic attack
188694    feel like ending would feel better suffering d...
20015     sometimes feel like cannot anymoremaybe partia...
141904    lost live anymorei going kill 2 hour 15 mintue...
127158    release new netflix film drama september liter...
Name: text, Length: 31312, dtype: object

## 3.3  Tokenizacion

In [ ]:
X_clean = X_clean.apply(word_tokenize)
X_clean_test = X_clean_test.apply(word_tokenize)
X_clean.head()

170783    [really, hate, lifei, 13, years, old, really, ...
60450     [ending, tonightim, leaving, many, friends, pe...
159181    [yo, anyone, chat, 14m, emo, spotify, addict, ...
63934     [someone, knocked, fun, one, parents, knocked,...
90090     [know, scared, screw, something, bestie, f, re...
Name: text, dtype: object

## 3.4 Lematizacion/Stemming

In [ ]:
# Funciones tomadas de https://www.kdnuggets.com/2018/03/text-data-preprocessing-walkthrough-python.html

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    words = lemmatize_verbs(words)
    words = stem_words(words)
    return words

In [ ]:
%%time
X_clean = X_clean.apply(stem_and_lemmatize) 
X_clean_test = X_clean_test.apply(stem_and_lemmatize)
X_clean.head()

CPU times: user 46.1 s, sys: 461 ms, total: 46.5 s
Wall time: 46.6 s


170783    [real, hat, life, 13, year, old, real, hat, li...
60450     [end, tonightim, leav, many, friend, pet, girl...
159181    [yo, anyon, chat, 14m, emo, spot, addict, long...
63934     [someon, knock, fun, on, par, knock, door, eve...
90090     [know, scar, screw, someth, besty, f, real, cl...
Name: text, dtype: object

# IV. Implementacion de modelos

## Representacion vectorial de datos

Dado que es necesario realizar una representacion vectorial del texto para su posterior procesamiento se seleccionan 3 tipos de representacion diferentes, para cada modelo se van a utilizar las 3 representaciones vectoriales para poder seleccionar el mejor resultado posible 


*   Matriz de Frecuencia
*   Representacion termino documento (TFIDF)
*   Matriz binaria

Inicialmente se combinan todos los tokens que componen cada sentencia en un string para poder crear representaciones matriciales 

In [ ]:
X_clean = X_clean.apply(lambda x: ' '.join(map(str, x)))
X_clean_test = X_clean_test.apply(lambda x: ' '.join(map(str, x)))
X_clean.head(3)

170783      real hat life 13 year old real hat lif want die
60450     end tonightim leav many friend pet girlfriend ...
159181    yo anyon chat 14m emo spot addict longest play...
Name: text, dtype: object

### Matriz frecuencia

In [ ]:
# También, se puede realizar el conteo de las palabras presentes.
count = CountVectorizer()
X_count = count.fit_transform(X_clean)
print(X_count.shape)

(31312, 40792)


In [ ]:
X_count_test=count.transform(X_clean_test)
print(X_count_test.shape)

(7828, 40792)


### tfidf

In [ ]:
#Se puede realizar la transformación Term-frecuency times inverse Document-frecuency.
tf_idf = TfidfVectorizer()
X_tf_idf = tf_idf.fit_transform(X_clean)
print(X_tf_idf.shape)

(31312, 40792)


In [ ]:
X_TfIdf_test=tf_idf.transform(X_clean_test)
print(X_TfIdf_test.shape)

(7828, 40792)


### Binaria

In [ ]:
# Se transforma el texto con una transformación binaria 1 a 1.
binary = CountVectorizer(binary=True, lowercase = False)
X_binary = binary.fit_transform(X_clean)
print(X_binary.shape)

(31312, 40792)


In [ ]:
X_binary_test=binary.transform(X_clean_test)
print(X_binary_test.shape)

(7828, 40792)


In [ ]:
train_datasets={"binary":X_binary,
                "count":X_count,
                "tfidf":X_tf_idf}
test_datasets={"binary":X_binary_test,
                "count":X_count_test,
                "tfidf":X_TfIdf_test}

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues,size=(10,10)):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')



    fig, ax = plt.subplots(figsize=size)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
print(X_binary.shape)

(31312, 40792)


In [ ]:
print(y_train.shape)

(31312,)


## 4.1 Algoritmo 1: Arbol de decision

In [ ]:
estimators = [
        ('classifier', DecisionTreeClassifier(random_state=0))          
    ]

parameters = {
              'classifier__criterion':['gini','entropy'],
              'classifier__max_depth':[30,40,50,60,70],
            }

pipe_tree = Pipeline(estimators)

In [ ]:
%%time
best_trees={}
for key in train_datasets:
  grid_search_tree = GridSearchCV(pipe_tree, parameters, scoring='f1', cv=3, verbose=3)
  with tf.device('/device:GPU:0'):
    grid_search_tree.fit(train_datasets[key],y_train)
    best_trees[key]=grid_search_tree.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END classifier__criterion=gini, classifier__max_depth=30;, score=0.816 total time=   5.5s
[CV 2/3] END classifier__criterion=gini, classifier__max_depth=30;, score=0.829 total time=   7.3s
[CV 3/3] END classifier__criterion=gini, classifier__max_depth=30;, score=0.829 total time=   5.4s
[CV 1/3] END classifier__criterion=gini, classifier__max_depth=40;, score=0.811 total time=   6.3s
[CV 2/3] END classifier__criterion=gini, classifier__max_depth=40;, score=0.827 total time=   6.1s
[CV 3/3] END classifier__criterion=gini, classifier__max_depth=40;, score=0.824 total time=   6.2s
[CV 1/3] END classifier__criterion=gini, classifier__max_depth=50;, score=0.810 total time=   6.6s
[CV 2/3] END classifier__criterion=gini, classifier__max_depth=50;, score=0.823 total time=   8.5s
[CV 3/3] END classifier__criterion=gini, classifier__max_depth=50;, score=0.826 total time=   6.7s
[CV 1/3] END classifier__criterion=gini, classif

In [ ]:
print(y_test.shape)

(7828,)


In [ ]:
model=best_trees[key]['classifier']
y_pred=model.predict(test_datasets[key])

In [ ]:
print(y_pred.shape)

(7828,)


In [ ]:
tree_results={}
for key in best_trees:
  model=best_trees[key]['classifier']
  y_pred=model.predict(test_datasets[key])
  acc=accuracy_score(y_true=y_test, y_pred=y_pred)
  prec=precision_score(y_true=y_test, y_pred=y_pred)
  recall=recall_score(y_true=y_test, y_pred=y_pred)
  f1=f1_score(y_true=y_test, y_pred=y_pred)
  tree_results[key]=[best_trees[key]['classifier'], acc, prec, recall, f1]

In [ ]:
df_results=pd.DataFrame.from_dict(tree_results, orient='index')
mapping = {df_results.columns[1]: 'Accuracy', df_results.columns[2]: 'Precision', df_results.columns[3]: 'Recall', df_results.columns[4]: 'F1', df_results.columns[0]: 'Tree type'}
df_results = df_results.rename(columns=mapping)
df_results

,Tree type,Accuracy,Precision,Recall,F1
binary,"DecisionTreeClassifier(max_depth=30, random_st...",0.856541,0.859660,0.801408,0.829513
count,"DecisionTreeClassifier(max_depth=30, random_st...",0.858201,0.864111,0.800235,0.830947
tfidf,"DecisionTreeClassifier(max_depth=30, random_st...",0.855391,0.857233,0.801408,0.828381


In [ ]:
tree_results

{'binary': [DecisionTreeClassifier(max_depth=30, random_state=0),
  0.8565406234031682,
  0.8596601636249214,
  0.801408037547668,
  0.8295126764839836],
 'count': [DecisionTreeClassifier(max_depth=30, random_state=0),
  0.8582013285641288,
  0.8641114982578397,
  0.8002346729246114,
  0.8309473042948524],
 'tfidf': [DecisionTreeClassifier(max_depth=30, random_state=0),
  0.85539090444558,
  0.8572325070599309,
  0.801408037547668,
  0.8283808368708308]}

## 4.2 Algoritmo 2: KNN


In [ ]:
estimators = [
        ('classifier', KNeighborsClassifier())          
    ]

parameters = {
              'classifier__weights':['uniform','distance'],
              'classifier__n_neighbors':[5,6,7,8,9,10,11,12],
              'classifier__p':[1,2],
            }

pipe_knn = Pipeline(estimators)

In [ ]:
%%time
best_knn={}
for key in train_datasets:
  grid_search_knn = GridSearchCV(pipe_knn, parameters, scoring='f1', cv=3, verbose=3)
  with tf.device('/device:GPU:0'):
    grid_search_knn.fit(train_datasets[key],y_train)
    best_knn[key]=grid_search_knn.best_estimator_

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END classifier__n_neighbors=5, classifier__p=1, classifier__weights=uniform;, score=0.620 total time= 1.1min
[CV 2/3] END classifier__n_neighbors=5, classifier__p=1, classifier__weights=uniform;, score=0.644 total time= 1.1min
[CV 3/3] END classifier__n_neighbors=5, classifier__p=1, classifier__weights=uniform;, score=0.648 total time= 1.0min
[CV 1/3] END classifier__n_neighbors=5, classifier__p=1, classifier__weights=distance;, score=0.620 total time= 1.1min
[CV 2/3] END classifier__n_neighbors=5, classifier__p=1, classifier__weights=distance;, score=0.645 total time= 1.0min
[CV 3/3] END classifier__n_neighbors=5, classifier__p=1, classifier__weights=distance;, score=0.648 total time= 1.0min
[CV 1/3] END classifier__n_neighbors=5, classifier__p=2, classifier__weights=uniform;, score=0.620 total time=  13.6s
[CV 2/3] END classifier__n_neighbors=5, classifier__p=2, classifier__weights=uniform;, score=0.644 total time=

In [ ]:
knn_results={}
for key in best_knn:
  model=best_knn[key]['classifier']
  y_pred=model.predict(test_datasets[key])
  acc=accuracy_score(y_true=y_test, y_pred=y_pred)
  prec=precision_score(y_true=y_test, y_pred=y_pred)
  recall=recall_score(y_true=y_test, y_pred=y_pred)
  f1=f1_score(y_true=y_test, y_pred=y_pred)
  knn_results[key]=[best_knn[key]['classifier'], acc, prec, recall, f1]

In [ ]:
df_results=pd.DataFrame.from_dict(knn_results, orient='index')
mapping = {df_results.columns[1]: 'Accuracy', df_results.columns[2]: 'Precision', df_results.columns[3]: 'Recall', df_results.columns[4]: 'F1', df_results.columns[0]: 'Tree type'}
df_results = df_results.rename(columns=mapping)
df_results

## 4.3 Algoritmo 3

In [ ]:
estimators = [
        ('classifier', SVC(C=1,gamma="scale", kernel="rbf" ))          
    ]

parameters = {
              'classifier__C':[0.1, 1],
              'classifier__gamma':['scale', 'auto'],
              'classifier__kernel':['linear','rbf'],
            }

pipe_svm = Pipeline(estimators)

In [ ]:
%%time
best_svm={}
for key in train_datasets:
  grid_search_svm = GridSearchCV(pipe_svm, parameters, scoring='f1', cv=3, verbose=3)
  with tf.device('/device:GPU:0'):
    grid_search_svm.fit(train_datasets[key],y_train)
    best_svm[key]=grid_search_svm.best_estimator_

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=linear;, score=0.895 total time= 1.3min
[CV 2/3] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=linear;, score=0.902 total time= 1.3min
[CV 3/3] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=linear;, score=0.896 total time= 1.3min
[CV 1/3] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=rbf;, score=0.844 total time= 2.9min
[CV 2/3] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=rbf;, score=0.851 total time= 3.0min
[CV 3/3] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=rbf;, score=0.847 total time= 3.1min
[CV 1/3] END classifier__C=0.1, classifier__gamma=auto, classifier__kernel=linear;, score=0.895 total time= 1.5min
[CV 2/3] END classifier__C=0.1, classifier__gamma=auto, classifier__kernel=linear;, score=0.902 total time= 1.5min
[CV 3/3] END classifier

In [ ]:
svm_results={}
for key in best_svm:
  model=best_svm[key]['classifier']
  y_pred=model.predict(test_datasets[key])
  acc=accuracy_score(y_true=y_test, y_pred=y_pred)
  prec=precision_score(y_true=y_test, y_pred=y_pred)
  recall=recall_score(y_true=y_test, y_pred=y_pred)
  f1=f1_score(y_true=y_test, y_pred=y_pred)
  svm_results[key]=[best_svm[key]['classifier'], acc, prec, recall, f1]

In [ ]:
df_results=pd.DataFrame.from_dict(svm_results, orient='index')
mapping = {df_results.columns[1]: 'Accuracy', df_results.columns[2]: 'Precision', df_results.columns[3]: 'Recall', df_results.columns[4]: 'F1', df_results.columns[0]: 'Tree type'}
df_results = df_results.rename(columns=mapping)
df_results

# V. Seleccion del mejor modelo

Con base en los resultados experimentales es posible determinar que el mejor modelo de todos los evaluados tiene los siguientes parametros :



*   Representacio vectorial
*   Modelo:SVC
*   C:
*   Gamma: 


Se va a hacer una ejecucion de este modelo sobre la totalidad de los datos provistos. Inicialmente se hacen los mismos pasos de preprocesamiento descritos en el numeral 3.2. Dado que estos ya se describieron en detalle aqui se van a ejecutar como un unico bloquellamando las funciones definidas en esa seccion

In [ ]:
##notese que aqui se esta utilizando el df completo y no el fragmento 'data small' utilizado para la seleccion del modelo
train, test = train_test_split(df, test_size=0.2, random_state=42)

##aca se redefine ytrain y ytest
X_train, y_train = train['text'], train['class']
X_test, y_test = test['text'], test['class']
X_full= X_train.apply(contractions.fix) 
X_full_test= X_test.apply(contractions.fix) 
X_full=X_full.apply(to_lowercase)
X_full_test=X_full_test.apply(to_lowercase)
X_full=X_full.apply(remove_punctuation)
X_full_test=X_full_test.apply(remove_punctuation)
X_full=X_full.apply(remove_stopwords)
X_full_test=X_full_test.apply(remove_stopwords)
X_full = X_full.apply(word_tokenize)
X_full_test = X_full_test.apply(word_tokenize)
X_full = X_full.apply(stem_and_lemmatize) 
X_full_test = X_full_test.apply(stem_and_lemmatize)


In [ ]:
binary = CountVectorizer(binary=True, lowercase = False)
X_binary = binary.fit_transform(X_full)
print(X_binary.shape)

In [ ]:
X_binary_test=binary.transform(X_full_test)
print(X_binary_test.shape)

In [ ]:
##usan parametros del mejor modelo y se ejecuta
estimators = [
        ('classifier', SVC(C=1,gamma="scale", kernel="rbf" ))          
    ]

parameters = {
              'classifier__C':[0.1, 1],
              'classifier__gamma':['scale', 'auto'],
              'classifier__kernel':['linear','rbf'],
            }
pipe_svm = Pipeline(estimators)

In [ ]:
grid_search_svm = GridSearchCV(pipe_svm, scoring='f1', cv=3, verbose=3)
with tf.device('/device:GPU:0'):
    grid_search_svm.fit(X_full,y_train)

# VI. Evaluacion de resultados